# Covid-19 Dashboard

In [17]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [18]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [19]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("timeseries.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [20]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=rawdata['data']
    # Extract all dates from the datalist and sort them in order
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    # Function to create a panda date from the date string
    # Create a panda date for the first date in the series
    startdate=parse_date(dates[0])
    # Create a panda date for the last date in the series
    enddate=parse_date(dates[-1])
    # Create a list of dates from the start date to the end date
    index=pd.date_range(startdate, enddate, freq='D')
    # Create a timeseries of dates with the columns
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    # Update the timeseries with data for the columns ['cases', 'hospital', 'deaths']
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                timeseriesdf.loc[date, column]=value
    # Fill in any empty values with 0s
    timeseriesdf.fillna(0.0, inplace=True)
    return timeseriesdf

df=wrangle_data(rawdata = jsondata) # df is the dataframe for plotting

In [21]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = ['areaType=overview']
    # Define the structure of the data you want retruned from the API
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate"
        }
    # Create an API object
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    return(timeseries) # return data read from the API

In [22]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True
    
apibutton=wdg.Button(
    description='Refresh Data', # you may want to change this...
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to refresh the data in the graph",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='refresh'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

## Time Series Graph of Cases, Hospitalisations and Deaths

Cases relates to daily case relates

Hospital relates to new hospital admissions

Deaths relates to cumulative deaths within 28 days of positive test by death date per 100k resident population

In [23]:

#def plot_graph(column):
#    df[column].plot()

# Create a function to flick between log scale and columns displayed on the graph
def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
#whichcolumn=wdg.Dropdown(
#    options=['cases', 'hospital', 'deaths'],
#    value='cases',
#    description='Data: ',
#    disabled=False,
#)

# Create a widget for the user to select columns
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

# Create a button to allow users to flick between log and linear displays
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back"""
    #global graph
    #graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

    
controls=wdg.VBox([series, scale, apibutton])
    
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

form=wdg.HBox([graph, controls])    
display(form)

**Author and Copyright Notice** (C) Guy Hunt, 2020 (g.p.w.hunt@se20.qmul.ac.uk). All rights reserved.: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*